### TODO:

###### Build up fundamental model:
-Test speeds vs traditional methods  
-Energy distance test for distributions  
-Check distributions of variables within households, compare to naive method w/borysov model somehow  
-Test making epsilon the same distribution as the actual posteriors in the model

###### Differentiate from the GenSynth paper:
-Travel diaries  
-Method/heuristic/rules for checking large number of attributes  
-New models; Disentangled VAE/GAN  
-Model population changes over time RNN?  
-Behavioral variables  

###### They suggest:
-Incorporate RNN to generate trip chains (time, location, mode, purpose)  
-Use GAN/other method to generate less inconsistencies  
-Address next stage of re-sampling to get future populations  

In [1]:
# Each input to training the model is a person's daily trip diary
# Inputs; day of week, characteristics of person/hh
# Outputs; trip purpose, mode, duration, distance
# How to include Time of Day?
# Timesteps could either be hours in the day, or trips in a chain?
    # If a timestep is a trip, add the time of departure to the output variables
    
# Timestep is a trip
# Each timestep has departure time, duration, distance, mode, and purpose (y)
# Each trip has person/hh variables, day of week, and previous timestep info (x)

## Import Libraries and Datasets

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.preprocessing as skpre
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
# Load in the persons PUMS dataset for WA state
t_df = pd.read_csv('data/NHTS/nhts_survey/trippub.csv')
p_df = pd.read_csv('data/NHTS/nhts_survey/perpub.csv')
h_df = pd.read_csv('data/NHTS/nhts_survey/hhpub.csv')

## Choose Variables and Preprocess

In [4]:
# Filter to desired variables (numeric then categorical)
#TRIPPURP = simplified why/from
nhts_data_t = t_df[['TDCASEID','HOUSEID','PERSONID','TDAYDATE','TRAVDAY','TDTRPNUM','STRTTIME','TRVLCMIN','TRPMILES','TRPTRANS','WHYFROM','WHYTO']]
nhts_data_p = p_df[['HOUSEID','PERSONID','R_AGE','TIMETOWK','EDUC','R_SEX','OCCAT']]
nhts_data_h = h_df[['HOUSEID','HHSIZE','HHFAMINC','HHVEHCNT']]
del t_df
del p_df
del h_df
nhts_data = pd.merge(nhts_data_t, nhts_data_h, on='HOUSEID', how='left')
nhts_data = pd.merge(nhts_data, nhts_data_p, on=['HOUSEID', 'PERSONID'], how='left')

# Give each set of daily trips a unique chain id (each will be an input to model)
chain_ids = nhts_data.groupby(['TDAYDATE','HOUSEID','PERSONID']).ngroup().values
nhts_data = nhts_data.drop(labels=['TDAYDATE','TDCASEID','HOUSEID','PERSONID'], axis=1)
nhts_data['CHAINID'] = chain_ids

# Remove NA values and check n before/after
print(f"Dataset n={len(nhts_data)} pre-cleaning")
nan_indices = list((nhts_data < 0).any(axis=1))
nan_ids = chain_ids[nan_indices]
nhts_data = nhts_data[~(nhts_data['CHAINID'].isin(nan_ids))]
chain_ids = nhts_data['CHAINID'].values
print(f"Dataset n={len(nhts_data)} post-cleaning")

Dataset n=923572 pre-cleaning
Dataset n=405590 post-cleaning


In [5]:
# Record constants for some characteristics of the dataset before coding vars
CAT_IDX = 18
STATIC_IDX = [0,1,2,3,4,5,6,7]
DIM_DYNAMIC = nhts_data_t.shape[1] - 3  # P/H/T id
DIM_STATIC = nhts_data_p.shape[1] + nhts_data_h.shape[1] - 3  # P/H/T id
DIM_TOTAL = nhts_data.shape[1]
VAR_NAMES = nhts_data.columns
NUM_SAMPLES = len(pd.unique(chain_ids))

# Split categorical data into OHE vars, save num classes per variable
dummies_list = []
for x in range(CAT_IDX, DIM_TOTAL):
    dummies = nhts_data.iloc[:,x]
    dummies = pd.get_dummies(dummies, prefix=f"{nhts_data.columns[x]}_")
    dummies_list.append(dummies)
CAT_LENGTHS = [x.shape[1] for x in dummies_list]
print(f"Categorical variable class lengths: {CAT_LENGTHS}")

Categorical variable class lengths: []


In [6]:
# Final data frame after encoding OHE
model_data_df = nhts_data.iloc[:,:CAT_IDX]
for ohe_var in dummies_list:
    model_data_df = pd.concat([model_data_df, ohe_var], axis=1)
DIM_MANIFEST = model_data_df.shape[1]

# Preview data that will be fed into model
model_data_df

,TRAVDAY,TDTRPNUM,STRTTIME,TRVLCMIN,TRPMILES,TRPTRANS,WHYFROM,WHYTO,HHSIZE,HHFAMINC,HHVEHCNT,R_AGE,TIMETOWK,EDUC,R_SEX,OCCAT,CHAINID
2,2,1,700,120,84.004,6,3,1,3,7,5,66,120,3,1,2,46938
3,2,2,1800,150,81.628,6,1,3,3,7,5,66,120,3,1,2,46938
6,5,1,1115,15,8.017,6,1,3,2,8,4,55,10,5,1,4,46940
7,5,2,2330,10,8.017,6,3,1,2,8,4,55,10,5,1,4,46940
8,5,1,550,15,3.395,4,1,16,1,10,2,45,30,5,2,4,24626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923567,3,1,810,27,1.168,1,1,3,1,10,0,52,20,5,1,4,93638
923568,3,2,1320,8,0.238,1,3,16,1,10,0,52,20,5,1,4,93638
923569,3,3,1415,5,0.238,1,16,3,1,10,0,52,20,5,1,4,93638
923570,3,4,1820,25,0.867,1,3,12,1,10,0,52,20,5,1,4,93638


In [7]:
# Standardize the input data from -1 to 1 for numerical variables
scaler = skpre.StandardScaler()
model_data = model_data_df.drop('CHAINID', axis=1).values
model_data[:,:CAT_IDX] = scaler.fit_transform(model_data[:,:CAT_IDX])

# Remove rows with numerical outliers (3 < standard deviations)
print(f"Dataset n={len(model_data)} pre-numerical-outliers")
outlier_indices = np.where(np.any(model_data[:,:CAT_IDX] > 3, axis=1))
outlier_ids = chain_ids[outlier_indices]
to_remove_indices = ~(model_data_df['CHAINID'].isin(outlier_ids))
model_data_df = model_data_df[to_remove_indices]
model_data = model_data[to_remove_indices]
chain_ids = model_data_df['CHAINID'].values
print(f"Dataset n={len(model_data)} post-numerical-outliers")

# Separate chain ids into train/test data
chain_ids = pd.unique(chain_ids)
train_idx = round(len(chain_ids)*.9)
train_data = chain_ids[0:train_idx]
test_data = chain_ids[train_idx:len(chain_ids)]

# Filter model data into train/test data
train_data = model_data[model_data_df['CHAINID'].isin(train_data)]
test_data = model_data[model_data_df['CHAINID'].isin(test_data)]

Dataset n=405590 pre-numerical-outliers
Dataset n=345658 post-numerical-outliers


## Set Parameters and Define Model

In [8]:
# Hyperparameters
BATCH_SIZE = 64
EPOCHS = 100
LEARN_RATE = 0.001
RHO = 0.9
LATENT_DIM = 5
HIDDEN_DIM = 100
KL_WEIGHT = 0.5

In [ ]:
# Recurrent


In [ ]:
# Static


In [ ]:
# Decoder
decoder_inputs = keras.Input(shape=(LATENT_DIM + LEN_HH,))
decoder_x = layers.Dense(HIDDEN_DIM, activation="tanh")(decoder_inputs)
decoder_x = layers.Dense(HIDDEN_DIM, activation="tanh")(decoder_x)
decoder_num_outputs = layers.Dense(CAT_IDX, activation="linear")(decoder_x)
decoder_cat_outputs = []
for var_length in CAT_LENGTHS:
    layer = layers.Dense(var_length, activation="softmax")(decoder_x)
    decoder_cat_outputs.append(layer)
decoder = keras.Model(decoder_inputs, [decoder_num_outputs, decoder_cat_outputs], name="decoder")
decoder.summary()

In [ ]:
# Define custom loss function for combined numerical and categorical data
def get_reconstruction_loss(data, reconstruction, CAT_IDX, CAT_LENGTHS):

    # Mean squared error for numerical variables
    data_num = data[:,:CAT_IDX]
    reconstruction_num = reconstruction[0]
    loss_num = keras.losses.mean_squared_error(data_num, reconstruction_num)
    loss_num = tf.reduce_mean(loss_num)
    
    # Categorical cross entropy for categorical variables
    loss_list = []
    current = CAT_IDX
    for i, x in enumerate(CAT_LENGTHS):
        data_cat = data[:,current:(current + x)]
        reconstruction_cat = reconstruction[1][i]
        loss = keras.losses.categorical_crossentropy(data_cat, reconstruction_cat, from_logits=False)
        loss = tf.reduce_mean(loss)
        loss_list.append(loss)
        current += x
    loss_cat = tf.reduce_sum(loss_list)

    # Return both losses; they are combined in the model
    return (loss_num, loss_cat)

def get_kl_loss(z_mean, z_log_var):
    kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
    kl_loss = tf.reduce_mean(kl_loss)
    kl_loss = -kl_loss
    return kl_loss

In [ ]:
# Loss metric recorder
loss_tracker = keras.metrics.Mean(name="loss")

# Variational Autoencoder
class VAE(keras.Model):
    def __init__(self, encoder, decoder, CAT_IDX, CAT_LENGTHS, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.CAT_IDX = CAT_IDX
        self.CAT_LENGTHS = CAT_LENGTHS

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            # Get latent vars from the encoder; feed to decoder and get sampled manifest variables
            z_mean, z_log_var, z = encoder(data[:,:MANIFEST_DIM])
            decoder_input = tf.concat([z, data[:,MANIFEST_DIM:]], axis=1)
            reconstruction = decoder(decoder_input)

            # Get loss between input values and reconstruction
            reconstruction_loss_num, reconstruction_loss_cat = get_reconstruction_loss(
                data,
                reconstruction,
                self.CAT_IDX,
                self.CAT_LENGTHS
            )
            reconstruction_loss = tf.add(reconstruction_loss_num, reconstruction_loss_cat)

            # Get Kullback Leidler loss between normal distribution and actual for latent variables
            kl_loss = get_kl_loss(z_mean, z_log_var)
            kl_loss = kl_loss * KL_WEIGHT

            # Combine into single loss term
            total_loss = reconstruction_loss + kl_loss

        # Get new gradients given the loss and take another step (update weights)
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        # Record the loss metrics
        loss_tracker.update_state(total_loss)

        return {
            "total_loss": loss_tracker.result(),
            "reconstruction_loss": reconstruction_loss,
            "reconstruction_loss_num": reconstruction_loss_num,
            "reconstruction_loss_cat": reconstruction_loss_cat,
            "kl_loss": kl_loss,
        }

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs[:,:MANIFEST_DIM])
        decoder_input = tf.concat([z, inputs[:,MANIFEST_DIM:]], axis=1)
        reconstruction = self.decoder(decoder_input)
        return reconstruction

    @property
    def metrics(self):
        return [loss_tracker]

## Training

In [ ]:
# Train
vae = VAE(encoder, decoder, CAT_IDX, CAT_LENGTHS)
vae.compile(optimizer=keras.optimizers.RMSprop(learning_rate=LEARN_RATE, rho=RHO))
history = vae.fit(train_data, epochs=EPOCHS, batch_size=BATCH_SIZE) #add callbacks=[callback] if debugging needed

In [ ]:
# Plot model loss/training progress
plt.plot(history.history['total_loss'])
plt.plot(history.history['reconstruction_loss_num'])
plt.plot(history.history['reconstruction_loss_cat'])
plt.plot(history.history['kl_loss'])
plt.title("Training History")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["total","rec_num","rec_cat","kl"], loc="upper right")
plt.show()

## Testing

In [ ]:
# Get latent vars from the encoder; feed to decoder and get sampled manifest variables
z_mean, z_logvar, z = vae.encoder.predict(test_data[:,:MANIFEST_DIM])

# Record the posterior trained distributions for z
latent_means = []
latent_vars = []

# Determine the average values for the mean/logvariance of the latent variables
for i in range(0, LATENT_DIM):
    epsilon = np.random.normal(loc=0, scale=1, size=1000)
    avg_mean = np.mean(z_mean[:,i])
    latent_means.append(avg_mean)
    avg_var = np.exp(np.mean(z_logvar[:,i]))
    latent_vars.append(avg_var)
    print(f"Latent Variable: {i}")
    print(f"Mean: {avg_mean}")
    print(f"Variance: {np.exp(avg_var)}\n")
    samples = avg_mean + (avg_var * epsilon)
    plt.hist(samples, bins=50)
    plt.show()

In [ ]:
# Draw predictions from test data
results = vae.predict(test_data)
loss_num, loss_cat = get_reconstruction_loss(test_data, results, CAT_IDX, CAT_LENGTHS)
print(f"Numerical Variable Loss: {loss_num}")
print(f"Categorical Variable Loss: {loss_cat}")

In [ ]:
# Transform numeric results back to real variable values
results_num = scaler_pers.inverse_transform(results[0])
results_df = pd.DataFrame(results_num)

# Transform categorical results back to real variable values
for x in results[1]:
    result = np.argmax(x, axis=1) + 1
    results_df[f"{x}"] = result

# Add back original variables names to the results
results_df.columns = VAR_NAMES

In [ ]:
# Transform numeric test data back to real variable values
test_data_num = scaler_pers.inverse_transform(test_data[:,:CAT_IDX])
test_data_df = pd.DataFrame(test_data_num)

# Transform categorical test data back to real variable values
current = CAT_IDX
for x in CAT_LENGTHS:
    test_data_cat = test_data[:,current:(current + x)]
    test_data_cat = np.argmax(test_data_cat, axis=1) + 1
    test_data_df[f"{x}"] = test_data_cat
    current += x

# Add back original variables names to the test data
test_data_df.columns = VAR_NAMES

In [ ]:
# Show distributions of the resulting numerical variables
for col_idx in range(0, CAT_IDX):
    results_data_plt = results_df.iloc[:,col_idx]
    test_data_plt = test_data_df.iloc[:,col_idx]

    plt.hist(results_data_plt, bins=100)
    plt.xlim(min(test_data_plt),max(test_data_plt))
    plt.title(f"var {col_idx} - reconstructed distribution")
    plt.show()

    plt.hist(test_data_plt, bins=100)
    plt.xlim(min(test_data_plt),max(test_data_plt))
    plt.title(f"var {col_idx} - prior distribution")
    plt.show()

In [ ]:
# Show distributions of the resulting categorical variables
for col_idx in range(CAT_IDX, VAR_DIM):
    results_data_plt = results_df.iloc[:,col_idx]
    test_data_plt = test_data_df.iloc[:,col_idx]
    
    plt.hist(results_data_plt)
    plt.xlim(min(test_data_plt),max(test_data_plt))
    plt.title(f"var {col_idx} - reconstructed distribution")
    plt.show()
    
    plt.hist(test_data_plt)
    plt.xlim(min(test_data_plt),max(test_data_plt))
    plt.title(f"var {col_idx} - prior distribution")
    plt.show()

## Generating Synthetic Population

In [ ]:
# Use list of households from the test data (in future can be generated with separate vae)
x = pd.DataFrame(test_data[:,MANIFEST_DIM:]).reset_index(drop=True) # Scaled hh_input values
y = model_data_df.iloc[train_idx:][['NP']].reset_index(drop=True) # Unscaled number of persons value
z = pd.concat([x,y],axis=1)
z.columns = ['HINCP','NP','VEH','SIZE']

# Multiply the inputs by the number of persons per household (hh of size 3 becomes 3 rows with same scaled hh inputs)
z = z.reindex(z.index.repeat(z['SIZE']))
z = z[['HINCP','NP','VEH']].values

# Generate random normal sample to represent each latent variable, for each row (different person per row)
epsilon = np.random.normal(loc=0, scale=1, size=(len(z), LATENT_DIM))
inputs = np.concatenate((epsilon, z), axis=-1)

# Generate persons; each person has unique latent input, plus shared hh inputs with their household
results = vae.decoder.predict(inputs)

In [ ]:
# Transform numeric results back to real variable values
results_num = scaler_pers.inverse_transform(results[0])
results_df = pd.DataFrame(results_num)

# Transform categorical results back to real variable values
for x in results[1]:
    result = np.argmax(x, axis=1) + 1
    results_df[f"{x}"] = result

# Add back original variables names to the results
results_df.columns = VAR_NAMES
results_df

In [ ]:
# Show distributions of the resulting numerical variables
for col_idx in range(0, CAT_IDX):
    results_data_plt = results_df.iloc[:,col_idx]
    test_data_plt = test_data_df.iloc[:,col_idx]

    plt.hist(results_data_plt, bins=100)
    plt.xlim(min(test_data_plt),max(test_data_plt))
    plt.title(f"var {col_idx} - reconstructed distribution")
    plt.show()

    plt.hist(test_data_plt, bins=100)
    plt.xlim(min(test_data_plt),max(test_data_plt))
    plt.title(f"var {col_idx} - prior distribution")
    plt.show()

In [ ]:
# Show distributions of the resulting categorical variables
for col_idx in range(CAT_IDX, VAR_DIM):
    results_data_plt = results_df.iloc[:,col_idx]
    test_data_plt = test_data_df.iloc[:,col_idx]
    
    plt.hist(results_data_plt)
    plt.xlim(min(test_data_plt),max(test_data_plt))
    plt.title(f"var {col_idx} - reconstructed distribution")
    plt.show()
    
    plt.hist(test_data_plt)
    plt.xlim(min(test_data_plt),max(test_data_plt))
    plt.title(f"var {col_idx} - prior distribution")
    plt.show()